In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import scale
from sklearn.feature_selection import f_classif, SelectKBest
from sklearn.model_selection import GridSearchCV
import category_encoders as ce
from sklearn.metrics import accuracy_score

In [133]:
X_df = pd.read_csv('/Users/ksmith/Documents/Code/DS1/Unit2_Project/DS_Unit2_Project/train_features.csv')
pd.set_option('display.max_columns', 100)
X_df.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [134]:
y_df = pd.read_csv('/Users/ksmith/Documents/Code/DS1/Unit2_Project/DS_Unit2_Project/train_labels.csv')
y_df.head()

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional


In [135]:
X_test_df= pd.read_csv('/Users/ksmith/Documents/Code/DS1/Unit2_Project/DS_Unit2_Project/test_features.csv')
X_test_df.head()

# Hold on to this one for later

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,50785,0.0,2013-02-04,Dmdd,1996,DMDD,35.290799,-4.059696,Dinamu Secondary School,0,Internal,Magoma,Manyara,21,3,Mbulu,Bashay,321,True,GeoData Consultants Ltd,Parastatal,NaN,True,2012,other,other,other,parastatal,parastatal,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,other,other
1,51630,0.0,2013-02-04,Government Of Tanzania,1569,DWE,36.656709,-3.309214,Kimnyak,0,Pangani,Kimnyak,Arusha,2,2,Arusha Rural,Kimnyaki,300,True,GeoData Consultants Ltd,VWC,TPRI pipe line,True,2000,gravity,gravity,gravity,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
2,17168,0.0,2013-02-01,NaN,1567,NaN,34.767863,-5.004344,Puma Secondary,0,Internal,Msatu,Singida,13,2,Singida Rural,Puma,500,True,GeoData Consultants Ltd,VWC,P,NaN,2010,other,other,other,vwc,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,other,other
3,45559,0.0,2013-01-22,Finn Water,267,FINN WATER,38.058046,-9.418672,Kwa Mzee Pange,0,Ruvuma / Southern Coast,Kipindimbi,Lindi,80,43,Liwale,Mkutano,250,NaN,GeoData Consultants Ltd,VWC,NaN,True,1987,other,other,other,vwc,user-group,unknown,unknown,soft,good,dry,dry,shallow well,shallow well,groundwater,other,other
4,49871,500.0,2013-03-27,Bruder,1260,BRUDER,35.006123,-10.950412,Kwa Mzee Turuka,0,Ruvuma / Southern Coast,Losonga,Ruvuma,10,3,Mbinga,Mbinga Urban,60,NaN,GeoData Consultants Ltd,Water Board,BRUDER,True,2000,gravity,gravity,gravity,water board,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe


In [136]:
#So date doesn't get dropped in next step

X_df['date_recorded'] = pd.to_datetime(X_df['date_recorded'])
X_test_df['date_recorded'] = pd.to_datetime(X_test_df['date_recorded'])

X_df['YearMonth'] = X_df['date_recorded'].map(lambda x: 100*x.year + x.month)
X_test_df['YearMonth'] = X_test_df['date_recorded'].map(lambda x: 100*x.year + x.month)

X_df['Year'] = X_df['date_recorded'].map(lambda x: x.year)
X_test_df['Year'] = X_test_df['date_recorded'].map(lambda x: x.year)

X_df['Month'] = X_df['date_recorded'].map(lambda x: x.month)
X_test_df['Month'] = X_test_df['date_recorded'].map(lambda x: x.month)

In [137]:
max_cardinality = 50

high_cardinality = [col for col in X_df.select_dtypes(exclude=np.number)
                   if X_df[col].nunique() > max_cardinality]

X_df = X_df.drop(columns=high_cardinality)
X_test_df = X_test_df.drop(columns=high_cardinality)

X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 35 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
gps_height               59400 non-null int64
longitude                59400 non-null float64
latitude                 59400 non-null float64
num_private              59400 non-null int64
basin                    59400 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
population               59400 non-null int64
public_meeting           56066 non-null object
recorded_by              59400 non-null object
scheme_management        55523 non-null object
permit                   56344 non-null object
construction_year        59400 non-null int64
extraction_type          59400 non-null object
extraction_type_group    59400 non-null object
extraction_type_class    59400 non-null object
ma

In [138]:
X_df['water_quality'] = X_df['water_quality'].replace('unknown', 'unknown_water_quality')
oh_wq = pd.get_dummies(X_df['water_quality'])
X_df = X_df.drop(columns ='water_quality')
X_df = X_df.join(oh_wq)

X_test_df['water_quality'] = X_test_df['water_quality'].replace('unknown', 'unknown_water_quality')
t_oh_wq = pd.get_dummies(X_test_df['water_quality'])
X_test_df = X_test_df.drop(columns ='water_quality')
X_test_df = X_test_df.join(t_oh_wq)

In [139]:
X_df['quantity'] = X_df['quantity'].replace('unknown', 'unknown_quantity')
oh_wq = pd.get_dummies(X_df['quantity'])
X_df = X_df.drop(columns ='quantity')
X_df = X_df.join(oh_wq)

X_test_df['quantity'] = X_test_df['quantity'].replace('unknown', 'unknown_quantity')
t_oh_wq = pd.get_dummies(X_test_df['quantity'])
X_test_df = X_test_df.drop(columns ='quantity')
X_test_df = X_test_df.join(t_oh_wq)

In [140]:
X_df['waterpoint_type_group'] = X_df['waterpoint_type_group'].replace('other', 'other_waterpoint_type_group')
oh_wq = pd.get_dummies(X_df['waterpoint_type_group'])
X_df = X_df.drop(columns ='waterpoint_type_group')
X_df = X_df.join(oh_wq)

X_test_df['waterpoint_type_group'] = X_test_df['waterpoint_type_group'].replace('other', 'other_waterpoint_type_group')
t_oh_wq = pd.get_dummies(X_test_df['waterpoint_type_group'])
X_test_df = X_test_df.drop(columns ='waterpoint_type_group')
X_test_df = X_test_df.join(t_oh_wq)

In [141]:
X_df['source_type'] = X_df['source_type'].replace('unknown', 'unknown_source_type')
X_df['source_type'] = X_df['source_type'].replace('dam', 'dam_source_type')
oh_st = pd.get_dummies(X_df['source_type'])
X_df = X_df.drop(columns ='source_type')
X_df = X_df.join(oh_st)

X_test_df['source_type'] = X_test_df['source_type'].replace('unknown', 'unknown_source_type')
X_test_df['source_type'] = X_test_df['source_type'].replace('dam', 'dam_source_type')
t_oh_wq = pd.get_dummies(X_test_df['source_type'])
X_test_df = X_test_df.drop(columns ='source_type')
X_test_df = X_test_df.join(t_oh_wq)

In [142]:
#X_df['basin'].value_counts()
oh_b = pd.get_dummies(X_df['basin'])
X_df = X_df.drop(columns ='basin')
X_df = X_df.join(oh_b)

t_oh_b = pd.get_dummies(X_test_df['basin'])
X_test_df = X_test_df.drop(columns ='basin')
X_test_df = X_test_df.join(t_oh_b)

In [143]:
#X_df['public_meeting'].value_counts().nlargest(25)

oh_pm = pd.get_dummies(X_df['public_meeting'])
X_df = X_df.drop(columns ='public_meeting')
X_df = X_df.join(oh_pm)

t_oh_pm = pd.get_dummies(X_test_df['public_meeting'])
X_test_df = X_test_df.drop(columns ='public_meeting')
X_test_df = X_test_df.join(t_oh_pm)

In [144]:
#X_df['scheme_management'].value_counts().nlargest(25)

oh_sm = pd.get_dummies(X_df['scheme_management'])
X_df = X_df.drop(columns ='scheme_management')
X_df = X_df.join(oh_sm)

t_oh_sm = pd.get_dummies(X_test_df['scheme_management'])
X_test_df = X_test_df.drop(columns ='scheme_management')
X_test_df = X_test_df.join(t_oh_sm)

In [145]:
#X_df['permit'].value_counts().nlargest(25)

X_df['permit'] = X_df['permit'].map({True: 1, False: 0, np.nan: -1})
X_test_df['permit'] = X_test_df['permit'].map({True: 1, False: 0, np.nan: -1})
X_test_df['None'] = 0

#X_df.describe(include='all')

In [146]:
#X_df['recorded_by'].value_counts().nlargest(25)

oh_rb = pd.get_dummies(X_df['recorded_by'])
X_df = X_df.drop(columns ='recorded_by')
X_df = X_df.join(oh_rb)

t_oh_rb = pd.get_dummies(X_test_df['recorded_by'])
X_test_df = X_test_df.drop(columns ='recorded_by')
X_test_df = X_test_df.join(t_oh_rb)

In [147]:
#X_df['extraction_type'].value_counts().nlargest(25)

X_df['extraction_type'] = X_df['extraction_type'].replace('other', 'other_extraction_type')
oh_et = pd.get_dummies(X_df['extraction_type'])
X_df = X_df.drop(columns ='extraction_type')
X_df = X_df.join(oh_et)

X_test_df['extraction_type'] = X_test_df['extraction_type'].replace('other', 'other_extraction_type')
t_oh_et = pd.get_dummies(X_test_df['extraction_type'])
X_test_df = X_test_df.drop(columns ='extraction_type')
X_test_df = X_test_df.join(t_oh_et)

X_test_df['other - mkulima/shinyanga'] = 0

In [148]:
#X_df['management_group'].value_counts().nlargest(25)

X_df['management_group'] = X_df['management_group'].replace('other', 'other_management_group')
X_df['management_group'] = X_df['management_group'].replace('unknown', 'unknown_management_group')
oh_et = pd.get_dummies(X_df['management_group'])
X_df = X_df.drop(columns ='management_group')
X_df = X_df.join(oh_et)

X_test_df['management_group'] = X_test_df['management_group'].replace('other', 'other_management_group')
X_test_df['management_group'] = X_test_df['management_group'].replace('unknown', 'unknown_management_group')
t_oh_et = pd.get_dummies(X_test_df['management_group'])
X_test_df = X_test_df.drop(columns ='management_group')
X_test_df = X_test_df.join(t_oh_et)

In [149]:
#X_df['payment_type'].value_counts().nlargest(25)

X_df['payment_type'] = X_df['payment_type'].replace('other', 'other_payment_type')
X_df['payment_type'] = X_df['payment_type'].replace('unknown', 'unknown_payment_type')
oh_et = pd.get_dummies(X_df['payment_type'])
X_df = X_df.drop(columns ='payment_type')
X_df = X_df.join(oh_et)

X_test_df['payment_type'] = X_test_df['payment_type'].replace('other', 'other_payment_type')
X_test_df['payment_type'] = X_test_df['payment_type'].replace('unknown', 'unknown_payment_type')
t_oh_et = pd.get_dummies(X_test_df['payment_type'])
X_test_df = X_test_df.drop(columns ='payment_type')
X_test_df = X_test_df.join(t_oh_et)

In [154]:
X_df.select_dtypes(exclude=np.number) #all of these features have other variables with very similar values

,region,extraction_type_group,extraction_type_class,management,payment,quality_group,quantity_group,source,source_class,waterpoint_type
0,Iringa,gravity,gravity,vwc,pay annually,good,enough,spring,groundwater,communal standpipe
1,Mara,gravity,gravity,wug,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe
2,Manyara,gravity,gravity,vwc,pay per bucket,good,enough,dam,surface,communal standpipe multiple
3,Mtwara,submersible,submersible,vwc,never pay,good,dry,machine dbh,groundwater,communal standpipe multiple
4,Kagera,gravity,gravity,other,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe
5,Tanga,submersible,submersible,vwc,pay per bucket,salty,enough,other,unknown,communal standpipe multiple
6,Shinyanga,swn 80,handpump,vwc,never pay,good,enough,machine dbh,groundwater,hand pump
7,Shinyanga,nira/tanira,handpump,wug,unknown,milky,enough,shallow well,groundwater,hand pump
8,Tabora,india mark ii,handpump,vwc,never pay,salty,seasonal,machine dbh,groundwater,hand pump
9,Kagera,nira/tanira,handpump,vwc,never pay,good,enough,shallow well,groundwater,hand pump


In [102]:
X_df.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,permit,construction_year,YearMonth,Year,Month,coloured,fluoride,fluoride abandoned,milky,salty,salty abandoned,soft,unknown_water_quality,dry,enough,insufficient,seasonal,unknown_quantity,cattle trough,communal standpipe,dam,hand pump,improved spring,other_waterpoint_type_group,borehole,dam_source_type,other,rainwater harvesting,river/lake,shallow well,spring,Internal,Lake Nyasa,Lake Rukwa,Lake Tanganyika,Lake Victoria,Pangani,Rufiji,Ruvuma / Southern Coast,Wami / Ruvu,False,True,Company,None,Other,Parastatal,Private operator,SWC,Trust,VWC,WUA,WUG,Water Board,Water authority,GeoData Consultants Ltd,afridev,cemo,climax,gravity,india mark ii,india mark iii,ksb,mono,nira/tanira,other - mkulima/shinyanga,other - play pump,other - rope pump,other - swn 81,other_extraction_type,submersible,swn 80,walimi,windmill
count,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.0,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,0.602626,1300.652475,201196.542306,2011.921667,4.375640,0.008249,0.003367,0.000286,0.013535,0.081751,0.005707,0.855522,0.031582,0.105152,0.558687,0.254697,0.068182,0.013283,0.001953,0.582912,0.000118,0.294411,0.013199,0.107407,0.201162,0.011044,0.004680,0.038636,0.174697,0.283232,0.286549,0.131061,0.085606,0.041313,0.108283,0.172525,0.150505,0.134276,0.075640,0.100791,0.085101,0.858771,0.017862,0.000017,0.012896,0.028283,0.017896,0.001633,0.001212,0.619411,0.048535,0.087643,0.046263,0.053081,1.0,0.029798,0.001515,0.000539,0.450842,0.040404,0.001650,0.023822,0.048232,0.137273,0.000034,0.001431,0.007593,0.003855,0.108249,0.080202,0.061785,0.000808,0.001970
std,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,0.585123,951.620547,94.990697,0.958758,3.029247,0.090450,0.057929,0.016915,0.115552,0.273987,0.075330,0.351577,0.174887,0.306751,0.496548,0.435694,0.252060,0.114484,0.044148,0.493082,0.010855,0.455781,0.114126,0.309633,0.400872,0.104508,0.068252,0.192728,0.379711,0.450572,0.452153,0.337469,0.279784,0.199015,0.310740,0.377840,0.357569,0.340952,0.264423,0.301055,0.279034,0.348261,0.132451,0.004103,0.112825,0.165781,0.132573,0.040378,0.034795,0.485536,0.214896,0.282778,0.210055,0.224197,0.0,0.170031,0.038896,0.023204,0.497582,0.196907,0.040585,0.152494,0.214259,0.344138,0.005803,0.037801,0.086805,0.061971,0.310697,0.271608,0.240766,0.028416,0.044338
min,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,200210.000000,2002.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [103]:
X_test_df.describe()

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,permit,construction_year,YearMonth,Year,Month,coloured,fluoride,fluoride abandoned,milky,salty,salty abandoned,soft,unknown_water_quality,dry,enough,insufficient,seasonal,unknown_quantity,cattle trough,communal standpipe,dam,hand pump,improved spring,other_waterpoint_type_group,borehole,dam_source_type,other,rainwater harvesting,river/lake,shallow well,spring,Internal,Lake Nyasa,Lake Rukwa,Lake Tanganyika,Lake Victoria,Pangani,Rufiji,Ruvuma / Southern Coast,Wami / Ruvu,False,True,Company,Other,Parastatal,Private operator,SWC,Trust,VWC,WUA,WUG,Water Board,Water authority,None,GeoData Consultants Ltd,afridev,cemo,climax,gravity,india mark ii,india mark iii,ksb,mono,nira/tanira,other - play pump,other - rope pump,other - swn 81,other_extraction_type,submersible,swn 80,walimi,windmill
count,14358.000000,14358.0000,14358.000000,1.435800e+04,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.0,14358.0,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000,14358.000000
mean,324.219996,653.6363,34.082414,-5.697584e+00,0.408971,15.156359,5.713052,187.055439,0.611436,1298.251985,201194.949018,2011.905349,4.414125,0.009124,0.002995,0.000139,0.013094,0.084691,0.005572,0.852277,0.032108,0.105725,0.556972,0.256233,0.068394,0.012676,0.002298,0.575289,0.000070,0.300251,0.012119,0.109974,0.204903,0.012049,0.004457,0.035869,0.172656,0.293286,0.276779,0.118331,0.081209,0.041789,0.110879,0.176557,0.149951,0.137066,0.075707,0.108511,0.088104,0.857222,0.019432,0.015949,0.027998,0.017760,0.001811,0.001393,0.613386,0.045550,0.087408,0.047639,0.056275,0.0,1.0,0.030018,0.001254,0.000557,0.429586,0.043182,0.002438,0.025770,0.051609,0.139783,0.001114,0.008218,0.003831,0.113595,0.083299,0.062474,0.000836,0.002438
std,2533.367778,688.2721,6.564449,2.947444e+00,8.231859,17.387588,9.794304,476.065978,0.574419,952.551852,95.892110,0.967263,3.048488,0.095085,0.054645,0.011802,0.113680,0.278432,0.074439,0.354837,0.176292,0.307496,0.496761,0.436567,0.252429,0.111875,0.047888,0.494316,0.008346,0.458383,0.109420,0.312868,0.403645,0.109109,0.066617,0.185969,0.377963,0.455284,0.447422,0.323011,0.273165,0.200113,0.313993,0.381306,0.357036,0.343929,0.264538,0.311035,0.283456,0.349858,0.138041,0.125284,0.164974,0.132083,0.042517,0.037298,0.486991,0.208513,0.282441,0.213009,0.230461,0.0,0.0,0.170643,0.035386,0.023599,0.495034,0.203273,0.049314,0.158453,0.221244,0.346774,0.033365,0.090285,0.061775,0.317330,0.276343,0.242023,0.028899,0.049314
min,0.000000,-57.0000,0.000000,-1.156459e+01,0.000000,1.000000,0.000000,0.000000,-1.000000,0.000000,200103.000000,2001.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000

In [164]:
X = X_df.select_dtypes('number')
X = X.drop(columns='id')
X = scale(X)

X_test_sc = X_test_df.select_dtypes('number')
X_test_sc = X_test_sc.drop(columns='id')
X_test_sc = scale(X_test_sc)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.75, test_size=0.25, random_state=42, shuffle=True)

X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_test_sc.shape

((44550, 93), (14850, 93), (44550, 1), (14850, 1), (14358, 93))

In [165]:
from sklearn.ensemble import RandomForestClassifier as RFC

rfc_b = RFC()

rfc_b.fit(X_train,y_train)
y_pred = rfc_b.predict(X_train)
print(accuracy_score(y_train,y_pred))
print(accuracy_score(y_test,rfc_b.predict(X_test)))

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.9803591470258137
0.7913131313131313


In [152]:
y_pred = rfc_b.predict(X_test_sc)

y_pred_df = pd.DataFrame(y_pred)

ids = pd.DataFrame(X_test_df['id'])

pass9 = ids.join(y_pred_df)

pass9.rename(columns={0:'status_group'}, 
                 inplace=True)

pass9.set_index('id')

pass9['status_group'] = pass9['status_group'].replace({0:'functional', 2:'non functional',1:'functional needs repair'})

In [153]:
pass9.to_csv(path_or_buf='/Users/ksmith/Documents/Code/DS1/Unit2_Project/DS_Unit2_Project/pass9.csv',index=False)

In [167]:
from sklearn.feature_selection import mutual_info_classif, SelectKBest

pipe = make_pipeline( 
    SelectKBest(mutual_info_classif), 
    RFC()
)

from sklearn.model_selection import GridSearchCV

param_grid = {
    'selectkbest__k': [10,25,50,75,93], 
    'randomforestclassifier__class_weight': [None,'balanced']
}

gs = GridSearchCV(pipe,param_grid=param_grid, cv=3,
                 scoring='accuracy')

gs.fit(X_train,y_train)

pd.DataFrame(gs.cv_results_).sort_values('rank_test_score')

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape 

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape 

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training sco

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__class_weight,param_selectkbest__k,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
8,21.589864,0.547601,0.044159,0.000170,balanced,75,{'randomforestclassifier__class_weight': 'bala...,0.789307,0.785993,0.785440,0.786914,0.001708,1,0.978619,0.978182,0.979294,0.978698,0.000457
9,21.709484,0.239874,0.050954,0.003908,balanced,93,{'randomforestclassifier__class_weight': 'bala...,0.781698,0.784310,0.791501,0.785836,0.004145,2,0.977912,0.978822,0.979462,0.978732,0.000636
3,21.471990,0.078704,0.043705,0.001568,None,75,"{'randomforestclassifier__class_weight': None,...",0.784863,0.787744,0.783285,0.785297,0.001846,3,0.979225,0.980067,0.980001,0.979764,0.000382
4,21.593454,0.169838,0.047085,0.000461,None,93,"{'randomforestclassifier__class_weight': None,...",0.787556,0.784108,0.781669,0.784444,0.002415,4,0.980134,0.978788,0.979630,0.979517,0.000555
2,21.506942,0.114272,0.036569,0.000845,None,50,"{'randomforestclassifier__class_weight': None,...",0.782708,0.783232,0.785036,0.783659,0.000997,5,0.977373,0.977811,0.980236,0.978474,0.001259
7,21.557383,0.136466,0.038287,0.000957,balanced,50,{'randomforestclassifier__class_weight': 'bala...,0.780284,0.783636,0.784565,0.782828,0.001839,6,0.976430,0.977643,0.979092,0.977722,0.001088
6,21.357264,0.092502,0.032202,0.000673,balanced,25,{'randomforestclassifier__class_weight': 'bala...,0.782035,0.778855,0.782814,0.781235,0.001712,7,0.975218,0.974343,0.974412,0.974658,0.000397
1,21.851390,0.246516,0.032351,0.000859,None,25,"{'randomforestclassifier__class_weight': None,...",0.780890,0.776364,0.783756,0.780337,0.003043,8,0.978181,0.976970,0.976869,0.977340,0.000596
0,21.750528,0.156334,0.030361,0.001036,None,10,"{'randomforestclassifier__class_weight': None,...",0.764730,0.768215,0.768941,0.767295,0.001838,9,0.973568,0.973569,0.970742,0.972626,0.001333
5,21.572419,0.261180,0.030176,0.000348,balanced,10,{'randomforestclassifier__class_weight': 'bala...,0.761969,0.758451,0.766112,0.762177,0.003131,10,0.967743,0.965926,0.967745,0.967138,0.000857


In [168]:
print('Best score:',gs.best_score_)
print('Parameters for best score:', gs.best_params_)

Best score: 0.7869135802469136
Parameters for best score: {'randomforestclassifier__class_weight': 'balanced', 'selectkbest__k': 75}


In [169]:
test_score = gs.score(X_test, y_test)
print('Test Score:', test_score)

Test Score: 0.7931313131313131


In [171]:
# Which features were selected?
X_names = X_df.select_dtypes('number')
X_names = X_names.drop(columns='id')

selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X_names.columns
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)

Features selected:
amount_tsh
gps_height
longitude
latitude
num_private
region_code
district_code
population
permit
construction_year
YearMonth
Month
fluoride
fluoride abandoned
milky
salty abandoned
soft
unknown_water_quality
dry
enough
insufficient
unknown_quantity
communal standpipe
dam
hand pump
improved spring
other_waterpoint_type_group
borehole
dam_source_type
rainwater harvesting
river/lake
shallow well
spring
Lake Nyasa
Lake Tanganyika
Lake Victoria
Pangani
Ruvuma / Southern Coast
Wami / Ruvu
False
True
Company
Parastatal
Private operator
VWC
WUA
WUG
Water Board
GeoData Consultants Ltd
climax
gravity
india mark ii
india mark iii
ksb
mono
nira/tanira
other - mkulima/shinyanga
other - swn 81
other_extraction_type
submersible
swn 80
walimi
windmill
commercial
other_management_group
parastatal
unknown_management_group
user-group
annually
monthly
never pay
on failure
other_payment_type
per bucket
unknown_payment_type

Features not selected:
Year
coloured
salty
seasonal
cattle troug

In [172]:
X = X_df.select_dtypes('number')
X = X.drop(columns=['id','Year','coloured','salty','seasonal','cattle trough',
                          'other','Internal','Lake Rukwa','Rufiji','None','Other',
                          'SWC','Trust','Water authority','afridev','cemo',
                          'other - play pump','other - rope pump'])
X = scale(X)

X_test_sc = X_test_df.select_dtypes('number')
X_test_sc = X_test_sc.drop(columns=['id','Year','coloured','salty','seasonal','cattle trough',
                          'other','Internal','Lake Rukwa','Rufiji','None','Other',
                          'SWC','Trust','Water authority','afridev','cemo',
                          'other - play pump','other - rope pump'])
X_test_sc = scale(X_test_sc)

X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=0.75, test_size=0.25, random_state=42, shuffle=True)

X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_test_sc.shape

((44550, 75), (14850, 75), (44550, 1), (14850, 1), (14358, 75))

In [173]:
rfc_b = RFC(class_weight='balanced')

rfc_b.fit(X_train,y_train)
y_pred = rfc_b.predict(X_train)
print(accuracy_score(y_train,y_pred))
print(accuracy_score(y_test,rfc_b.predict(X_test)))

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


0.9791021324354657
0.7916498316498316


In [174]:
y_pred = rfc_b.predict(X_test_sc)

y_pred_df = pd.DataFrame(y_pred)

ids = pd.DataFrame(X_test_df['id'])

pass10 = ids.join(y_pred_df)

pass10.rename(columns={0:'status_group'}, 
                 inplace=True)

pass10.set_index('id')

pass10['status_group'] = pass10['status_group'].replace({0:'functional', 2:'non functional',1:'functional needs repair'})

In [175]:
pass10.to_csv(path_or_buf='/Users/ksmith/Documents/Code/DS1/Unit2_Project/DS_Unit2_Project/pass10.csv',index=False)

#Got a .752 with this one

In [ ]:
pipe = make_pipeline( 
    SelectKBest(mutual_info_classif), 
    RFC()
)

from sklearn.model_selection import GridSearchCV

param_grid = {
    'selectkbest__k': [60,65,70,75], 
    'randomforestclassifier__class_weight': [None,'balanced']
}

gs = GridSearchCV(pipe,param_grid=param_grid, cv=3,
                 scoring='accuracy')

gs.fit(X_train,y_train)

pd.DataFrame(gs.cv_results_).sort_values('rank_test_score')

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:250: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params)
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
